# Create 3D city model

Creates a 3D city model from any set of 2D sources (without 3dfying).

In [54]:
import geopandas
import json
from tqdm.notebook import tqdm
import shapely


sources = [
    {
        "type": "Road",
        "path": "Toronto/roads.gpkg",
        "id_column": "guid",
        "lod": "1.0"
    },
    {
        "type": "Building",
        "path": "Toronto/buildings.gpkg",
        "id_column": "guid",
        "lod": "0"
    }
]

output = "toronto.json"

In [68]:
def create_cityobject(feature, vertices, obj_type=None, lod=None, geom_col="geometry"):
    """Create a city object from a GeoSeries"""
    
    if not "type" in feature and obj_type is None:
        raise KeyError("City object type is not defined!")

    atts = dict(feature)
    
    if "type" in atts:
        del atts["type"]
    del atts["geometry"]
    
    obj = {
        "type": feature["type"] if obj_type is None else obj_type,
        "attributes": atts,
        "geometry": []
    }
    
    geom, vertices = shape_to_geom(feature[geom_col], feature["lod"] if lod is None else lod, vertices)
    obj["geometry"].append(geom)
    
    return obj, vertices

def shape_to_geom(shape, lod, vertices):
    """Converts a shapely geometry to a CityJSON geometry"""
    
    if shape.geom_type == "Point":
        geom_type = "MultiPoint"
        b, v = point_to_geom(shape, len(vertices))
    elif shape.geom_type == "LineString":
        geom_type = "MultiLineString"
        b, v = linestring_to_geom(shape, len(vertices))
        b = [b]
    elif shape.geom_type == "MultiLineString":
        geom_type = "MultiLineString"
        b = []
        v = []
        for line in shape.geoms:
            b_temp, v_temp = linestring_to_geom(shape, len(vertices) + len(v))
            b.append(b_temp)
            v.extend(v_temp)
    elif shape.geom_type == "Polygon":
        geom_type = "MultiSurface"
        b, v = polygon_to_geom(shape, len(vertices))
        b = [b]
    elif shape.geom_type == "MultiPolygon":
        geom_type = "MultiSurface"
        b = []
        v = []
        for poly in shape.geoms:
            b_temp, v_temp = polygon_to_geom(poly, len(vertices) + len(v))
            b.append(b_temp)
            v.extend(v_temp)
    else:
        raise TypeError(f"Geometry type '{shape.geom_type}' not supported!")
    
    geom = {
        "type": geom_type,
        "lod": lod,
        "boundaries": b
    }
    
    v = [[c[0], c[1], 0] if len(c) == 2 else c for c in v]
    
    return (geom, v)

def point_to_geom(point, offset):
    """Returns the boundary indices and new vertices of Point"""
    
    indices = [offset]
    verts = [point.coords[0]]
    
    return (indices, verts)

def linestring_to_geom(line, offset, remove_last=False, reverse=False):
    """Returns the boundary indices and new vertices of a LineString
    
    This only returns the indices of one linestring, therefore it cannot be directly
    assigned to a geometry's boundary values.
    """
    
    coords = line.coords
    if remove_last:
        coords = coords[:-1]
    
    if reverse:
        coords = coords[::-1]
    
    indices = [offset + i for i in range(len(coords))]
    verts = [list(c) for c in coords]
    
    return (indices, verts)

def polygon_to_geom(polygon, offset):
    """Returns the boundary indices and new vertices of a MultiSurface based on the given polygon"""
    
    indices = []
    verts = []
    
    b, v = linestring_to_geom(polygon.exterior, offset, True, not polygon.exterior.is_ccw)
    
    indices.append(b)
    verts.extend(v)
    
    for hole in polygon.interiors:
        b, v = linestring_to_geom(hole, offset + len(verts), True, not hole.is_ccw)
    
        indices.append(b)
        verts.extend(v)
    
    return (indices, verts)

In [69]:
cm = {
    "type": "CityJSON",
    "version": "1.0",
    "CityObjects": {},
    "vertices": []
}

for source in sources:
    co_type = source["type"]
    objs = geopandas.read_file(source["path"])
    
    objs = objs.set_index(source["id_column"])

    for co_id in objs.index:
        obj, new_verts = create_cityobject(objs.loc[co_id], cm["vertices"], obj_type=co_type, lod=source["lod"])

        cm["CityObjects"][co_id] = obj
        cm["vertices"].extend(new_verts)

In [66]:
objs.iloc[0]["geometry"].geoms[0].exterior.is_ccw

False

In [70]:
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

with open(output, "w") as out:
    json.dump(cm, out, cls=NpEncoder)